# 1 Setup

## 1.1 Google Drive - Kaggle

In [ ]:
# Google drive setup
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install kaggle
!mkdir .kaggle

In [ ]:
import json
token = {"username":"argajadeepakshende","key":"5fab11c8416a4039725989fbef9ccbd2"}
with open('.kaggle/kaggle.json','w') as file: json.dump(token, file)

## 1.2 Kaggle Data Download

In [ ]:
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json /root/.kaggle/
!kaggle config set -n path -v /content

- path is now set to: /content


In [ ]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Found existing installation: kaggle 1.5.12
Uninstalling kaggle-1.5.12:
  Successfully uninstalled kaggle-1.5.12
     |████████████████████████████████| 1.7 MB 5.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 58 kB 3.2 MB/s             
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72857 sha256=1768180d57d61617d1d97a4b08965735093e1e03107367927a9cd13898e67cad
  Stored in directory: /root/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
Successfully built kaggle


In [ ]:
# download data
!kaggle competitions download -c 11785-fall2021-hw3p2

100% 2.34G/2.35G [00:12<00:00, 98.6MB/s]
100% 2.35G/2.35G [00:12<00:00, 209MB/s] 


In [ ]:
!unzip /content/competitions/11785-fall2021-hw3p2/11785-fall2021-hw3p2.zip

Archive:  /content/competitions/11785-fall2021-hw3p2/11785-fall2021-hw3p2.zip
  inflating: HW3P2_Data/dev.npy      
  inflating: HW3P2_Data/dev_labels.npy  
  inflating: HW3P2_Data/phoneme_list.py  
  inflating: HW3P2_Data/sample_submission.csv  
  inflating: HW3P2_Data/test.npy     
  inflating: HW3P2_Data/train.npy    
  inflating: HW3P2_Data/train_labels.npy  


In [ ]:
!ls HW3P2_Data/

dev_labels.npy	phoneme_list.py        test.npy		 train.npy
dev.npy		sample_submission.csv  train_labels.npy


## 1.3 Library Installations

Install [ctcdecode](https://github.com/parlance/ctcdecode)

In [ ]:
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .
%cd ..

Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 1102 (delta 16), reused 28 (delta 13), pack-reused 1063
Receiving objects: 100% (1102/1102), 780.91 KiB | 12.39 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 14051, done.        
remote: Counting objects: 100% (364/364), done.        
remote: Compressing objects: 100% (296/296), done.        
remote: Total 14051 (delta 10

Install [levenshtein distance calculation library](https://github.com/ztane/python-Levenshtein) 

In [ ]:
!pip install python-Levenshtein

     |████████████████████████████████| 50 kB 3.2 MB/s             
  Preparing metadata (setup.py) ... done
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149874 sha256=88207d60d988bfb68c9247b5edf50143782d2ba06448dff59ab8814a4c444633
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


## 1.4 Libraries & Setup

In [ ]:
import os
import sys
import time

import Levenshtein as lev

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pdb
import gc
from tqdm.notebook import trange, tqdm

import torch
import torch.nn as nn

from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Check if cuda is available and set device
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

num_workers = 8 if cuda else 0

print("Cuda = ", str(cuda), " with num_workers = ", str(num_workers),  " system version = ", sys.version)

Cuda =  True  with num_workers =  8  system version =  3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]


# 2 Data Loading

## 2.1 Load Data

In [ ]:
# load training and dev data
train_data = np.load('HW3P2_Data/train.npy', allow_pickle=True)
train_labels = np.load('HW3P2_Data/train_labels.npy', allow_pickle=True)

dev_data = np.load('HW3P2_Data/dev.npy', allow_pickle=True)
dev_labels = np.load('HW3P2_Data/dev_labels.npy', allow_pickle=True)

# load test data
test_data = np.load('HW3P2_Data/test.npy', allow_pickle=True)

In [ ]:
print(f'Train data: {train_data.shape}')
print(f'Train labels {train_labels.shape}')

print(f'Dev data: {dev_data.shape}')
print(f'Dev labels {dev_labels.shape}')

print(f'Test data: {test_data.shape}')

Train data: (14542,)
Train labels (14542,)
Dev data: (2200,)
Dev labels (2200,)
Test data: (2561,)


## 2.2 Custom Dataset Classes

In [ ]:
# Define dataset class
class MyDataSet(Dataset):
  # load the dataset
  def __init__(self, x, y):
    self.X = x
    self.Y = y

  # get number of items/rows in dataset
  def __len__(self):
    return len(self.Y)

  # get row item at some index
  def __getitem__(self, index):
    x = torch.as_tensor(self.X[index], dtype=torch.float32)
    y = torch.as_tensor(self.Y[index], dtype=torch.long)
    
    return x, y

  


In [ ]:
def collate_fn(batch):
    
    input = []
    target = []
    for x in batch:
      input.append(x[0])
      target.append(x[1])
    input_padded = pad_sequence(input, batch_first=True) 
    target_padded = pad_sequence(target, batch_first=True)
    input_len = torch.as_tensor([len(x) for x in input],dtype=torch.long)
    target_len = torch.as_tensor([len(x) for x in target],dtype=torch.long)
    return input_padded, target_padded, input_len, target_len

In [ ]:
# Define dataset class
class TestDataSet(Dataset):
  
  def __init__(self, x):
    self.X = x

  
  def __len__(self):
    return len(self.X) 

 
  def __getitem__(self, index):
    x = torch.as_tensor(self.X[index], dtype=torch.float32)
    return x,torch.as_tensor(np.array([-1]), dtype=torch.long)


## 2.3 Data Loaders

In [ ]:
batch_size = 32 # TODO: decide on batch size

# training data
train = MyDataSet(train_data, train_labels)
#train_args = .... # TODO: remember to use collate_fn
train_loader = DataLoader(train, shuffle=True, batch_size=batch_size, num_workers=8, collate_fn=collate_fn, pin_memory=True)

# validation data
dev = MyDataSet(dev_data, dev_labels)
#dev_args = .... # TODO: remember to use collate_fn
dev_loader = DataLoader(dev, shuffle=True, batch_size=batch_size, num_workers=8, collate_fn=collate_fn, pin_memory=True)

# test data
test = TestDataSet(test_data)
#test_args = .... # TODO: remember to use collate_fn
test_loader = DataLoader(test, shuffle=False, batch_size=batch_size, num_workers=8, collate_fn=collate_fn, pin_memory=True)

In [ ]:
train_data[0].shape

(1504, 40)

# 2 Model Building

## 2.1 Model Creation

In [ ]:
# TODO: Create model    
class LSTMModel(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, output_size):
    super(LSTMModel, self).__init__()
    self.cnn = torch.nn.Sequential(
            nn.Conv1d(input_size, hidden_size, 3, padding=1, bias=False),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(inplace=True))
    self.lstm = nn.LSTM(input_size=hidden_size,
                            hidden_size=hidden_size,
                            num_layers=4,
                            bias=True,
                            batch_first=True,
                            dropout=0.2, 
                            bidirectional=True)   
   
    
    self.linear = torch.nn.Sequential(
            nn.Linear(hidden_size*2, hidden_size),
            nn.Linear(hidden_size, hidden_size),
            nn.Linear(hidden_size, output_size))

    
  def forward(self, x, lengths): 
    x_cnn_in = x.permute(0, 2, 1) 
    x_lstm_in = self.cnn(x_cnn_in).permute(2, 0, 1) 
    x_packed_input = pack_padded_sequence(x_lstm_in, lengths, enforce_sorted=False)
    x_packed_out, hidden = self.lstm(x_packed_input)        
    x_out, out_len = pad_packed_sequence(x_packed_out, batch_first=True)       
   
    output = self.linear(x_out)
    output = output.log_softmax(2)   
    output = output.permute(1, 0, 2)
    
    return output, out_len

## 2.2 Model Initialization

In [ ]:
# create model
input_size = 40
hidden_size = 512
num_layers = 4
output_size = 42

model = LSTMModel(input_size, hidden_size, num_layers, output_size)
model = model.to(device)
print(model)

LSTMModel(
  (cnn): Sequential(
    (0): Conv1d(40, 512, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (lstm): LSTM(512, 512, num_layers=4, batch_first=True, dropout=0.2, bidirectional=True)
  (linear): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=42, bias=True)
  )
)


# 4 Model Training

## 4.0 Set Hyperparameters

In [ ]:
# Hyperparams
criterion = nn.CTCLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-3 , weight_decay=5e-6)
    
# You can add a LR scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=1, verbose=True)


## 4.1 Train Epoch

In [ ]:
# Train the model
def train(model, train_loader, criterion, optimizer,epoch):
  model.train()

  avg_loss = 0.0
  start = time.time()

  for batch_idx, (data, target, data_len, target_len) in enumerate(train_loader):
        optimizer.zero_grad()
        data = data.cuda() 
        target = target.cuda()
        data_len = data_len.cuda()
        target_len = target_len.cuda()

        output, out_data_len = model(data, data_len) 
        avg_loss = criterion(output, 
                         target, 
                         out_data_len, 
                         target_len) 
        avg_loss.backward()
        optimizer.step()
        if batch_idx % 50 == 0:
            print("Epoch: {}\tBatch: {}\tTimestamp: {}".format(epoch, batch_idx, time.time() - start))
        
  end = time.time()
  

## 4.2 CTC Decoding

In [ ]:
import sys
sys.path.append("HW3P2_Data")

from phoneme_list import PHONEME_MAP

In [ ]:
from ctcdecode import CTCBeamDecoder


def decode(output_prob, data_len, beamWidth):
    decoder = CTCBeamDecoder(labels=PHONEME_MAP, beam_width=1,
                            num_processes=os.cpu_count(), log_probs_input=True)
    output_prob = torch.permute(output_prob, (1,0,2)) 
    output, beam_scores, timesteps, out_seq_len = decoder.decode(output_prob, data_len) 
    decoded = []
    batch_size = output_prob.shape[0]
    for b in range(batch_size):
        decode = "".join([PHONEME_MAP[i] for i in output[b, 0, :out_seq_len[b][0]]])
        decoded.append(decode)      
        
    return decoded

In [ ]:
def to_string(target):
    return "".join([PHONEME_MAP[x] for x in target])

## 4.3 Validate Epoch

In [ ]:
def test(model, data_loader, epoch):
    model.eval()
    start_time = time.time()
    running_loss = 0.0
    running_dist = 0.0
    for batch_idx, (data, target, data_len, target_len) in enumerate(data_loader):
      data = data.cuda()
      target = target.cuda()
      data_len = data_len.cuda()
      target_len = target_len.cuda()
      optimizer.zero_grad()   
      output, out_data_len = model(data, data_len)
      loss = criterion(output,target,
                             out_data_len,
                             target_len)
            
      running_loss = running_loss + loss.item()
      decoded_string = decode(output, data_len, 1)
      target_string = []
      for i in target:
        target_string.append(to_string(i))
      for i in range(len(target_string)):
        dist = lev.distance(decoded_string[i].replace(" ", ""), target_string[i].replace(" ", ""))                    
        running_dist += dist
        running_dist /= (i+1)    
        if batch_idx % 50 == 0:
          print("Epoch: {}\tBatch: {}\tTimestamp: {}".format(epoch, batch_idx, time.time() - start_time))
            
            
      loss_per_sample = running_loss / len(data_loader)
        
      return loss_per_sample, running_dist

## 4.4 Run Epochs

# 5 Test Data

## 5.1 Make Predictions

In [ ]:
def predict(model, data_loader):
    model.eval()
    prediction = []
    prediction = np.asarray(prediction)   
    for batch_idx, (data, target, data_len, target_len) in enumerate(data_loader):
        data = data.cuda()
        target = target.cuda()
        data_len = data_len.cuda()
        target_len = target_len.cuda()
        output, out_data_len = model(data, data_len)        
        decoded_string = decode(output, data_len, 50)
        prediction = np.append(prediction, decoded_string)     
        
        
    return prediction

In [ ]:
for i in range(26):
  
  print("Epoch: {}".format(i))
  train(model, train_loader, criterion, optimizer, i)
  train_loss, train_dist = test(model, train_loader, i)
  print('Train_Loss: {:.4f}\tTrain_Dist: {:.4f}'.format(
         train_loss, train_dist))
  dev_loss, dev_dist = test(model, dev_loader, i)
  print('Dev_Loss: {:.4f}\tDev_Dist: {:.4f}'.format(
            dev_loss, dev_dist))
  scheduler.step(dev_loss)

        

In [ ]:
for i in range(2): 
  
  print("Epoch: {}".format(i))
  train(model, train_loader, criterion, optimizer, i)
  train_loss, train_dist = test(model, train_loader, i)
  print('Train_Loss: {:.4f}\tTrain_Dist: {:.4f}'.format(
         train_loss, train_dist))
  dev_loss, dev_dist = test(model, dev_loader, i)
  print('Dev_Loss: {:.4f}\tDev_Dist: {:.4f}'.format(
            dev_loss, dev_dist))

  scheduler.step(dev_loss)

        

In [ ]:

for i in range(7):
 
  print("Epoch: {}".format(i))
  train(model, train_loader, criterion, optimizer, i)
  train_loss, train_dist = test(model, train_loader, i)
  print('Train_Loss: {:.4f}\nTrain_Dist: {:.4f}'.format(
         train_loss, train_dist))
  dev_loss, dev_dist = test(model, dev_loader, i)
  print('Dev_Loss: {:.4f}\nDev_Dist: {:.4f}'.format(
            dev_loss, dev_dist))

  scheduler.step(dev_loss)

        

In [ ]:
model = torch.load('model_new.pt')

In [ ]:
# Hyperparams


criterion = nn.CTCLoss()
optimizer = optim.Adam(model.parameters(), lr=2.5e-4 , weight_decay=5e-6)
    
# You can add a LR scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=1, verbose=True)


In [ ]:
for i in range(1):
  
  print("Epoch: {}".format(i))
  train(model, train_loader, criterion, optimizer, i)
  train_loss, train_dist = test(model, train_loader, i)
  print('Train_Loss: {:.4f}\nTrain_Dist: {:.4f}'.format(
         train_loss, train_dist))
  dev_loss, dev_dist = test(model, dev_loader, i)
  print('Dev_Loss: {:.4f}\nDev_Dist: {:.4f}'.format(
            dev_loss, dev_dist))

  scheduler.step(dev_loss)

        

In [ ]:
torch.save(model,'model_new.pt')

In [ ]:
pred = predict(model, test_loader)
ids = np.array(range(len(pred)))
df = pd.DataFrame({"id" : ids, "Label" : pred})
df.to_csv("prediction.csv", index=False)

## 5.2 Save Predictions to csv File

## 5.3 Submit Predictions